# Prompt engineering on worst econ questions: answer verification, temperature, red/blue, context

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
from ollama_models import ollama_models
models = ollama_models()

In [3]:
models

['mistral-small3.2:24b-instruct-2506-q4_K_M']

In [4]:
model = models[0]

In [5]:
from load_saved_questions import load_saved_questions
from call_local_llm import call_local_llm

In [6]:
question = load_saved_questions([12348])[0]

In [7]:
    from community_forecast import community_forecast
    import pandas as pd
    from flatten_dict import flatten_dict
    from datetime import datetime
    from gather_research_and_set_prompt import gather_research_and_set_prompt
    from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
    from tqdm import tqdm
    import time

In [8]:
    print('START model', model, 'id', id)
    start_time = time.time()
    questions = [question]
    id = question.id_of_question
    id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}
    df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
    df['id_of_post'] = [q.id_of_post for q in questions]
    df['id_of_question'] = [q.id_of_question for q in questions]
    df['question_options'] = df['question_options'].apply(repr)
    df['today'] = datetime.now().strftime("%Y-%m-%d")
    df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)

START model mistral-small3.2:24b-instruct-2506-q4_K_M id <built-in function id>


In [9]:
    df1 = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title',
        'question_resolution_criteria', 'question_fine_print', 'question_type', 
         'question_description',
        'question_options', 'question_group_variable', 'question_question_weight',
        'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
        'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']].copy()

In [10]:
    df2, rag = gather_research_and_set_prompt(df1, True, model)

Loaded existing index from forecast_index.faiss
Index contains 5839 vectors at initialization


In [11]:
df3 = df2[['id_of_question', 'title',
        'question_resolution_criteria', 'question_type', 
         'question_description','crowd',
          'research', 'asknews', 'learning']].copy()

In [12]:
row = df3.iloc[0]

In [13]:
row

id_of_question                                                              12348
title                                    Will 3M file for bankruptcy before 2026?
question_resolution_criteria    This question will resolve as **Yes** if there...
question_type                                                              binary
question_description            3M Company, one of the 30 components making up...
crowd                                                                        0.01
research                        ## 1. Key Historical Trends and Current Status...
asknews                         Here are the relevant news articles:\n\n**3M s...
learning                                                                       []
Name: 0, dtype: object

In [14]:
for item, value in row.items():
    print(item.upper())
    print()
    print(value)
    print('===============================')
    print()

ID_OF_QUESTION

12348

TITLE

Will 3M file for bankruptcy before 2026?

QUESTION_RESOLUTION_CRITERIA

This question will resolve as **Yes** if there is an 8-K filing stating that the entity filing with the SEC under the Central Index Key (CIK) number 66740 has filed or intends to file a petition in bankruptcy court under Chapter 11 reorganization or Chapter 7 liquidation before January 1, 2026

QUESTION_TYPE

binary

QUESTION_DESCRIPTION

3M Company, one of the 30 components making up the Dow Jones Industrial Average and ranked #102 on the Fortune 500, is a multinational corporation based in Minnesota with approximately 95,000 full-time equivalent employees. It manufactures a wide range of consumable industrial products used every day in offices, homes, hospitals, schools, factories and other workplaces. Well-known brands include Scotch-Brite abrasives and cleaning supplies, 3M respirators, Scotchgard Protector, Scotch tapes, Post-it notes, and Scotchlite films.

In 2021, 3M had operat

In [15]:
row.question_resolution_criteria

'This question will resolve as **Yes** if there is an 8-K filing stating that the entity filing with the SEC under the Central Index Key (CIK) number 66740 has filed or intends to file a petition in bankruptcy court under Chapter 11 reorganization or Chapter 7 liquidation before January 1, 2026'

In [16]:
prompt2 = f"""
You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
{row.title}
```

```news
{row.asknews}

```research
{row.research}
```

The question defines an event and resolves as YES if the event occurs, otherwise NO. 
You are forecasting the probability of YES.
The event is described by the RESOLUTION CRITERIA.

```resolution_criteria
{row.question_resolution_criteria}
```

The last thing you write is your final answer as: "Probability: ZZ%", 0-100
"""

In [17]:
print(prompt2)


You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
Will 3M file for bankruptcy before 2026?
```

```news
Here are the relevant news articles:

**3M settles PFAS claims with New Jersey for $285 million**
3M has reached an agreement to settle all historical claims related to the Chambers Works site in Salem County, New Jersey, as well as broader state-wide claims regarding per- and polyfluoroalkyl substances (PFAS). The agreement, pending judicial approval, includes a commitment of approximately $210 million in present value before taxes for the Chambers Works litigation and an additional $75 million to address current and future claims from 2030 to 2050. According to InvestingPro's analysis, the stock appears slightly overvalued at current levels. 3M has provided PFAS to DuPont at the Chambers Works site until 2001 and has since taken steps to withdraw from PFAS m

In [18]:
answer_mistral = call_local_llm(prompt2, model)

answer_mistral2 = call_local_llm(prompt2, model)

answer_mistral3 = call_local_llm(prompt2, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 3.1879950364430747
START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 2.965402058760325
START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 2.9106903195381166


In [19]:
from extract_forecast import extract_probability_from_response_as_percentage_not_decimal

In [20]:
forecasts = [extract_probability_from_response_as_percentage_not_decimal(x)/100.0
             for x in [answer_mistral, answer_mistral2, answer_mistral3]]

In [21]:
forecasts

[0.05, 0.05, 0.05]

In [22]:
import numpy as np

In [23]:
forecast = float(np.median(forecasts))

In [24]:
forecast

0.05

In [25]:
prompt = f"""
Summarize the gist pf the rationale or thinking of the following answers from 3 different forecasters to a single problem. 

```forecast 1
{answer_mistral}
```

```forecast 2
{answer_mistral2}
```

```forecast 3
{answer_mistral3}
```

DO NOT REFER TO THE 3 FORECASTERS.  PRESENT THIS AS YOUR OWN THINKING, YOUR OWN RATIONALE.  Use as your final the median forecast which is

```median forecast
{forecast}
```
"""

In [26]:
rationale = call_local_llm(prompt, model)

START model mistral-small3.2:24b-instruct-2506-q4_K_M
model mistral-small3.2:24b-instruct-2506-q4_K_M minutes 1.1346849004427593


In [27]:
print(rationale)

The rationale for assessing the probability of 3M filing for bankruptcy before January 1, 2026, is based on a comprehensive evaluation of the company's financial health, legal context, and recent disclosures as of May 2025. Key factors supporting this assessment include:

1. **Consistent Profitability**: 3M has demonstrated steady profitability, indicating a stable financial position.
2. **Strong Cash Flows**: The company generates positive cash flow, which is crucial for meeting financial obligations and managing liabilities.
3. **No Imminent Distress Signals**: There are no indications or disclosures suggesting that 3M is experiencing financial distress or planning to file for bankruptcy.
4. **Legal Liabilities**: While legal liabilities pose a long-term risk, they do not currently threaten the company's solvency or operations.

Given these factors, the probability of 3M filing for bankruptcy before January 1, 2026, is assessed as very low, with a median forecast probability of 5%.


In [28]:
row.forecast = rationale

In [29]:
row.prediction = forecast

In [30]:
row.crowd

0.01

In [31]:
from error import error

In [32]:
error(row)

0.04